Role: 너는 금융 지식이 풍부한 Python 프로그래머야.

Action:
사용자로부터 한국 주식 종목 코드(6자리)를 input()으로 입력받는다.
requests와 pandas를 사용하여 네이버 금융에서 해당 종목의 현재가, 등락률, 시가총액, 거래량 정보를 가져온다. (공식 API가 없으므로 웹 크롤링 또는 비공식 경로를 사용)
가져온 데이터를 바탕으로 해당 종목의 현재 상태를 요약해서 브리핑해줘.

Context: 주피터 노트북 환경에서 실행될 코드를 작성해줘. 

Expectation:
외부 라이브러리 설치 명령문(!pip install ...)을 포함할 것.
입력은 input 출력은 깔끔한 Text 포맷으로만 제공할 것.

종목 코드가 유효하지 않을 경우에 대한 예외 처리를 포함할 것.

In [9]:
# =========================
# 1. 라이브러리 설치
# =========================
!pip install requests pandas

In [12]:

# =========================
# 2. 라이브러리 임포트
# =========================
import requests
import pandas as pd


# =========================
# 3. 안전한 숫자 변환 함수
# =========================
def to_int(value):
    return int(str(value).replace(",", ""))


def to_float(value):
    return float(str(value).replace(",", ""))


# =========================
# 4. 종목 코드 입력
# =========================
code = input("한국 주식 종목 코드(6자리)를 입력하세요: ").strip()

if not (code.isdigit() and len(code) == 6):
    raise ValueError("❌ 종목 코드는 6자리 숫자여야 합니다.")


# =========================
# 5. 네이버 금융 비공식 API
# =========================
url = f"https://polling.finance.naver.com/api/realtime/domestic/stock/{code}"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)

if response.status_code != 200:
    raise ConnectionError("❌ 네이버 금융 API 호출 실패")

data = response.json()

try:
    stock = data["datas"][0]

    name = stock["stockName"]
    price_int = to_int(stock["closePrice"])
    current_price = f"{price_int:,}"
    change_rate = f"{to_float(stock['fluctuationsRatio'])}%"
    volume = f"{to_int(stock['accumulatedTradingVolume']):,}"

    # =========================
    # 6. 시가총액 처리 (핵심)
    # =========================
    if "marketValue" in stock and stock["marketValue"]:
        market_cap = f"{to_int(stock['marketValue']):,}"
    elif "listedShares" in stock and stock["listedShares"]:
        # 시가총액 직접 계산
        market_cap = f"{price_int * to_int(stock['listedShares']):,}"
    else:
        market_cap = "정보 없음"

except (KeyError, IndexError, ValueError):
    raise ValueError("❌ 유효하지 않은 종목 코드이거나 데이터 형식 오류입니다.")


# =========================
# 7. pandas 정리
# =========================
df = pd.DataFrame([{
    "종목명": name,
    "현재가": current_price,
    "등락률": change_rate,
    "시가총액": market_cap,
    "거래량": volume
}])


# =========================
# 8. 텍스트 브리핑 출력
# =========================
print("\n================= 📊 주식 종목 브리핑 =================")
print(f"종목명   : {df.loc[0, '종목명']}")
print(f"현재가   : {df.loc[0, '현재가']} 원")
print(f"등락률   : {df.loc[0, '등락률']}")
print(f"시가총액 : {df.loc[0, '시가총액']}")
print(f"거래량   : {df.loc[0, '거래량']}")
print("======================================================")


한국 주식 종목 코드(6자리)를 입력하세요:  476800



================= 📊 주식 종목 브리핑 =================
종목명   : KODEX 한국부동산리츠인프라
현재가   : 4,890 원
등락률   : 0.51%
시가총액 : 정보 없음
거래량   : 1,252,918
